<a href="https://colab.research.google.com/github/pimang62/dacon-multimodal-vqa/blob/main/llava_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!rm -rf LLaVA
!rm -rf open.zip
!rm -rf data
!rm -rf transformers

In [ ]:
!pip install jsonlines

In [ ]:
%cd /content
!gdown https://drive.google.com/u/0/uc?id=1a9XB3r83ZCFWLOHBp8ooz3zQFl9rEIei&export=download

In [ ]:
%cd /content
!git clone https://github.com/dankang21/LLaVA.git

In [ ]:
%cd /content
!git clone https://github.com/huggingface/transformers

In [ ]:
import zipfile
import os

zip_file_path = '/content/open.zip'

extracted_folder = '/content/data/'

def extract_zip(zip_file, extract_to):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

try:
    extract_zip(zip_file_path, extracted_folder)
except Exception as e:
    print('file unzip error')

In [72]:
import csv
import json

with open('/content/data/test.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    data = list(reader)

In [ ]:
data = data[:1000]

In [74]:
que = [
    "What is this? please describe in detail.",
    "What is most important thing on this image?",
    "How do you feel inside this image?",
    "What is the relationship between central object and fringe?",
    "What will be happened in next?"
]

In [75]:
json_data = []
for row in data:
    id, image_id, question = row
    for i in range(len(que)):
      json_data.append({
          "question_id": id+'_'+str(i),
          "image": "/content/data/image/test/" + image_id + ".jpg",
          "text": que[i]
      })

jsonl_output_file = "/content/data/test.jsonl"

with open(jsonl_output_file, "w") as file:
    for obj in json_data:
        json.dump(obj, file)
        file.write("\n")

In [ ]:
filename_data = {}
for item in json_data:
    image = item['image']
    question_id = item['question_id'].rsplit('_', 1)[0]
    if image not in filename_data:
        filename_data[image] = {'question_id': question_id, 'image': image}

filename_json = json.dumps(list(filename_data.values()), indent=4)
print(filename_json)

In [ ]:
%cd /content/LLaVA
!pip install --upgrade pip
!pip install -e .
!pip install ninja
!pip install flash-attn --no-build-isolation

In [ ]:
%cd /content/LLaVA
!python3 /content/LLaVA/llava/eval/model_vqa.py \
    --model-path liuhaotian/LLaVA-Lightning-MPT-7B-preview \
    --question-file \
    /content/data/test.jsonl \
    --image-folder \
   /content/image/test \
    --answers-file \
    /content/data/result.jsonl

## Submission

In [ ]:
%cd /content

import pandas as pd

import json
grouped_data = {}
with open('/content/data/result.jsonl', 'r') as file:
  answer = []
  for line in file:
    info = json.loads(line)
    key = info['question_id'].rsplit('_', 1)[0]
    if key not in grouped_data:
      grouped_data[key] = info['text']
    else:
      grouped_data[key] += ' ' + info['text']

result_json = json.dumps(grouped_data, indent=2)
print(result_json)

In [82]:
output_json = filename_data.values()

for item in output_json:
    question_id = item["question_id"]
    if question_id in grouped_data:
        item["text"] = grouped_data[question_id]

In [ ]:
output = json.dumps(list(output_json), indent=2)
print(output)

In [ ]:
%cd /content/transformers
!pip install .
!pip install --upgrade .
!pip install sentencepiece

In [ ]:
import gc
gc.collect()

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_device

In [ ]:
model_name = 'google/pegasus-reddit_tifu'

tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
for item in output_json:
  batch = tokenizer.prepare_seq2seq_batch(item['text'], truncation=True, padding='longest', return_tensors="pt").to(torch_device)
  translated = model.generate(**batch)
  item['result'] = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [ ]:
for item in output_json:
    item.pop('question_id', None)
    item.pop('text', None)

In [ ]:
output = json.dumps(list(output_json), indent=2)
print(output)

In [ ]:
jsonl_output_file = "/content/data/image_qurated.jsonl"

with open(jsonl_output_file, "w") as file:
    for obj in output_json:
        json.dump(obj, file)
        file.write("\n")